# Hubbard Model

I'm planning on running VQE and Trotter simulation on a Hubbard Hamiltonian to better understand its structure. 

The Hubbard model is a simplification of the interactions of electrons in a solid. It can be used to explain how these interactions yield insulating, magnetic, and superconducting effects (though I don't understand any of this yet). 

In [2]:
import numpy as np
import scipy as sp
import scipy.optimize 
from utils import *

## Defining creation and annihilation operators 

Since we're describing electrons, the Hubbard Hamiltonian's creation and annihilation operators have the normal fermionic anticommutation relations, namely that for a fermion on site $j,k$ with spin $\sigma, \pi$, 

$$ \large \{ c_{j \sigma}, c^\dagger_{k \pi } \} = \delta_{jk} \delta_{\sigma \pi} \qquad \{ c^\dagger_{j\sigma}, c^\dagger_{k \pi} \}= \{ c_{j\sigma} c_{k\pi} \} = 0  $$

In [2]:
class Operator: 
    def __init__(self, _type, spin, qubit, dim):
        self.type = _type 
        self.spin = spin
        self.qubit = qubit
        self.dim = dim
        
class CreationOperator(Operator):
    def __init__(self, spin, qubit, dim):
        super(CreationOperator, self).__init__('creation', spin, qubit, dim)

class AnnihilationOperator(Operator):
    def __init__(self, spin, qubit, dim):
        super(AnnihilationOperator, self).__init__('annihilation', spin, qubit, dim)

In [3]:
c_1 = CreationOperator('down', 0, 5)
a_1 = AnnihilationOperator('up', 3, 5)

## Jordan-Wigner Transformation

In [4]:
def encode_JW(operator):
    if not issubclass(type(operator), Operator):
        raise TypeError("operator must be an Operator")
        
    result = np.array([[1.0]], dtype='complex128') 
    string_repr = ''
    # Notice the order of the tensor product: gate on 0th qubit is "leftmost" in product
    for i in range(operator.dim): 
        if i < operator.qubit: 
            result = NKron(result, Z)
            string_repr = string_repr + 'Z'
        elif i == operator.qubit and operator.type == 'creation': 
            result = NKron(result, (X -1j*Y) / 2)
            string_repr = string_repr + '-'
        elif i == operator.qubit and operator.type == 'annihilation': 
            result = NKron(result, (X +1j*Y) / 2) 
            string_repr = string_repr + '+'
        elif i > operator.qubit: 
            result = NKron(result, I) 
            string_repr = string_repr + 'I'
        else: 
            raise ValueError("Something is wrong with this code!")
    #print(string_repr)
    return result

## Defining the Hubbard Hamiltonian

Okay, now that we have those preliminaries out of the way, what is the Hubbard Hamiltonian? What's the motivation behind it? How is it useful? 

Answer these. 

The Hubbard Hamiltonian: 
$$ \large H = -t \sum_{<j, k>, \sigma} \Big( c^\dagger_{j\sigma} c_{k\sigma} + c^\dagger_{k \sigma} c_{j \sigma} \Big) + U \sum_j n_{j \uparrow} n_{j \downarrow} - \mu \sum_j \Big(n_{j\uparrow} + n_{j\downarrow} \Big) $$

The first term is kinetic energy, a fermion moving from one site to another. The symbol $<j, k>$ implies iterating over sites that are adjacent. 

The second term is interaction energy, additional energy for a doubly-occupied site. 

The third term is chemical potential, which controls the filling. **Hm, a lot of places don't have this third term (Wikipedia even). Why?**

### 2D Hubbard Hamiltonian on a Square Lattice 

Here is a model of a 2D Hubbard Hamiltontian on a square lattice from [1811.04476](https://arxiv.org/pdf/1811.04476.pdf). We'll represent it by a 2D array: 2D to get the square structure, and in each square `[i][j]`, a dictionary of 4 elements: 'c-up', 'c-down', 'a-up', and 'a-down'. 

![](images/2d_hubbard.png)

In [5]:
# Make sure N <= 10 or computer explodes!!!!!
N_X = 2
N_Y = 2
N = N_X * N_Y

In [6]:
# We use this to make iterating over neighbors easier 
# NOTE: in HubbardHamiltonian2D, we generate the JW transformation instead of the operator class
square_lattice = [[ {'i': i, 'j': j, 
                     'c-up': CreationOperator('UP', N_Y * i + j, N), 
                     'c-down': CreationOperator('DOWN', N_Y * i + j, N), 
                     'a-up': AnnihilationOperator('UP', N_Y * i + j, N), 
                     'a-down': AnnihilationOperator('DOWN', N_Y * i + j, N)
                    } for j in range(N_X)] for i in range(N_Y)]

In [7]:
square_lattice[0][1]

{'i': 0,
 'j': 1,
 'c-up': <__main__.CreationOperator at 0x7fbf978150b8>,
 'c-down': <__main__.CreationOperator at 0x7fbf97815128>,
 'a-up': <__main__.AnnihilationOperator at 0x7fbf97815160>,
 'a-down': <__main__.AnnihilationOperator at 0x7fbf97815198>}

In [8]:
# TODO: does c-up and c-down act on different qubits? I think they do. But then which qubits are "next" to them 
# in the lattice? 
# I think they need to be acting on different qubits (otherwise they're the same operator...) and even though I 
# make the spin DOWN qubits all after the spin UP qubits, as long as I keep this square lattice 2d array, I 
# should be fine I THINK. Let's try this. So spin DOWN acts on self.N + self.N_Y * i + j and dimension is 
# 2 * self.N

In [11]:
class HubbardHamiltonian2D: 
    def __init__(self, t, U, mu, N_X, N_Y):
        self.t = t 
        self.U = U 
        self.mu = mu 
        self.N_X = N_X 
        self.N_Y = N_Y
        self.N = N_X * N_Y 
        self.square_lattice = self._gen_square_lattice()
        self.ham_sets = [self._gen_hor_even_sum(), self._gen_hor_odd_sum(), 
                         self._gen_ver_even_sum(), self._gen_ver_odd_sum(), 
                         self._gen_interaction_energy()]
        self.hamiltonian = self._gen_hamiltonian()
        
    def _gen_square_lattice(self):
        # We use this to make iterating over neighbors easier 
        return [[ {'i': i, 'j': j, 
                   # LOOK AT COMMENT ABOVE!
#                    'c-up': encode_JW(CreationOperator('UP', self.N_Y * i + j, self.N)), 
#                    'c-down': encode_JW(CreationOperator('DOWN', self.N_Y * i + j, self.N)), 
#                    'a-up': encode_JW(AnnihilationOperator('UP', self.N_Y * i + j, self.N)), 
#                    'a-down': encode_JW(AnnihilationOperator('DOWN', self.N_Y * i + j, self.N))
                   'c-up': encode_JW(CreationOperator('UP', self.N_Y * i + j, 2 * self.N)), 
                   'c-down': encode_JW(CreationOperator('DOWN', self.N + self.N_Y * i + j, 2 * self.N)), 
                   'a-up': encode_JW(AnnihilationOperator('UP', self.N_Y * i + j, 2 * self.N)), 
                   'a-down': encode_JW(AnnihilationOperator('DOWN', self.N + self.N_Y * i + j, 2 * self.N))
                  } for j in range(self.N_X)] for i in range(self.N_Y)]
    
    def _gen_hamiltonian(self): 
        return (-self.t * (self.ham_sets[0] + self.ham_sets[1] + 
                           self.ham_sets[2] + self.ham_sets[3]) + 
                self.U * self.ham_sets[4])
    
    # I'm calculating the Hamiltonian like this because we'll be separating these terms later for the ansatz
    def _gen_hor_even_sum(self): 
        total = np.eye(2**(2 * self.N), dtype='complex128')
        for i in range(self.N_Y): 
            for j in range(self.N_X): 
                if j % 2 == 0 and j + 1 < self.N_X: 
                    site = self.square_lattice[i][j]
                    next_site = self.square_lattice[i][j+1]
                    total += NDot(site['c-up'], next_site['a-up'])
                    total += NDot(site['c-down'], next_site['a-down'])
                    total += NDot(next_site['c-up'], site['a-up'])
                    total += NDot(next_site['c-down'], site['a-down'])
        return total 
    
    def _gen_hor_odd_sum(self): 
        total = np.eye(2**(2 * self.N), dtype='complex128')
        for i in range(self.N_Y): 
            for j in range(self.N_X): 
                if j % 2 == 1 and j + 1 < self.N_X: 
                    site = self.square_lattice[i][j]
                    next_site = self.square_lattice[i][j+1]
                    total += NDot(site['c-up'], next_site['a-up'])
                    total += NDot(site['c-down'], next_site['a-down'])
                    total += NDot(next_site['c-up'], site['a-up'])
                    total += NDot(next_site['c-down'], site['a-down'])
        return total 
    
    def _gen_ver_even_sum(self):
        total = np.eye(2**(2 * self.N), dtype='complex128')
        for i in range(self.N_Y): 
            if i % 2 == 0 and i + 1 < self.N_Y: 
                for j in range(self.N_X): 
                    site = self.square_lattice[i][j]
                    next_site = self.square_lattice[i+1][j]
                    total += NDot(site['c-up'], next_site['a-up'])
                    total += NDot(site['c-down'], next_site['a-down'])
                    total += NDot(next_site['c-up'], site['a-up'])
                    total += NDot(next_site['c-down'], site['a-down'])
        return total 
    
    def _gen_ver_odd_sum(self): 
        total = np.eye(2**(2 * self.N), dtype='complex128')
        for i in range(self.N_Y): 
            if i % 2 == 1 and i + 1 < self.N_Y: 
                for j in range(self.N_X): 
                    site = self.square_lattice[i][j]
                    next_site = self.square_lattice[i+1][j]
                    total += NDot(site['c-up'], next_site['a-up'])
                    total += NDot(site['c-down'], next_site['a-down'])
                    total += NDot(next_site['c-up'], site['a-up'])
                    total += NDot(next_site['c-down'], site['a-down'])
        return total 
    
    def _gen_interaction_energy(self):
        total = np.eye(2**(2 * self.N), dtype='complex128')
        for i in range(self.N_Y): 
            for j in range(self.N_X): 
                site = self.square_lattice[i][j] 
                total += NDot(site['c-up'], site['a-up'], site['c-down'], site['a-down'])
        return total

In [12]:
x = HubbardHamiltonian2D(1, 2, 0, N_X, N_Y)

In [13]:
eig = np.linalg.eigh(x.hamiltonian)
index = np.argmin(eig[0])
print('Ground state energy: ', eig[0][index])
gstate = np.transpose(NKron(eig[1][:,index]))
gstate = gstate / np.linalg.norm(gstate)
print('Ground state: ', gstate)
print(gstate.shape)

Ground state energy:  -5.627213005296662
Ground state:  [[ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 1.38623404e-17+0.j]
 [ 1.84239949e-01+0.j]
 [ 2.59189359e-01+0.j]
 [ 0.00000000e+00+0.j]
 [ 2.59189359e-01+0.j]
 [ 0.00000000e+00+0.j]
 [ 5.55111512e-17+0.j]
 [ 0.00000000e+00+0.j]
 [ 2.85827558e-01+0.j]
 [-9.83340456e-18+0.j]
 [ 2.40473545e-17+0.j]
 [ 2.25953514e-19+0.j]
 [-2.79689362e-17+0.j]
 [-2.25953514e-19+0.j]
 [-8.44623034e-20+0.j]
 [ 0.00000000e+00+0.j]
 [ 4.72548419e-17+0.j]
 [ 2.59189359e-01+0.j]
 [ 1.84239949e-01+0.j]
 [-1.47241624e-17+0.j]
 [ 2.85827558e-01+0.j]
 [-4.97853663e-18+0.j]
 [ 1.95278013e-17+0.j]
 [ 2.25953514e-19+0.j]
 [ 2.59189359e-01+0.j]
 

In [14]:
np.dot(np.transpose(np.conj(gstate)), np.dot(x.hamiltonian, gstate))

array([[-5.62721301+0.j]])

## Variational Hamiltonian Ansatz

The VHA is an ansatz deeply connected to time-evolution of the system. It splits the Hamiltonian into sub-operators and then does time-evolution for those operators: 
$$\large U(\theta) = \prod_{k=1}^n \prod_{\alpha=1}^N \exp \Big( i\theta_{\alpha, k} H_{\alpha} \Big) $$
where $H_\alpha$ are the sub-Hamiltonians and $\theta$ are the parameters being optimized. 

**Question:** Is product from $k=1$ to $n$ meant to indicate multiple iterations of the algorithm? So are there only $N$ parameters, or are there $n \cdot N$? I think it's the former, because $\Big( \exp (i \theta_{1 k} H_1) \cdots \exp (i \theta_{N k} H_N) \Big)$ commutes with itself, so we can combine terms.  

**Question:** Why is this a good ansatz? It seems it only has access to states that are evolutions of the state we start with. Why is that a guarantee that it'll approximate the ground state? 

In [1811.04476](https://arxiv.org/pdf/1811.04476.pdf) they use $N=5$, splitting it as I did above: even and odd horizontal hopping terms, even and odd vertical horizontal terms, and the on-site interaction terms. 

In [15]:
def time_evo(ham, t): 
    # returns e^{i*t*ham}
    # scipy.linalg.expm() uses the Pade approximant; try to understand more of how it works because it seems 
    # pretty useful: https://en.wikipedia.org/wiki/Pad%C3%A9_approximant
    return scipy.linalg.expm(1j * t * ham)

def prod_time_evo(ham_sets, ts):
    result = 1 
    for pair in zip(ham_sets, ts): 
        result *= time_evo(pair[0], pair[1])
    return result

In [102]:
class VHA: 
    def __init__(self, hubbard, start_state): 
        self.ham_sets = hubbard.ham_sets 
        self.hamiltonian = hubbard.hamiltonian
        self.start_state = start_state
        self.start_params = np.array([.5 for i in range(5)])
            
    def optimize(self):  
        self.energies = []
        
        def energy(t_params): 
            unitary = prod_time_evo(self.ham_sets, np.real(t_params))
            state = np.dot(unitary, self.start_state)
            curr_energy = np.dot(np.transpose(np.conj(state)), np.dot(self.hamiltonian, state))[0][0]
            #print(curr_energy)
            self.energies.append(curr_energy)
            return curr_energy
                    
        self.sol = scipy.optimize.minimize(energy, self.start_params, method='Powell', 
                                           bounds=[(0, 3) for i in range(len(self.start_params))])

In [41]:
vha = VHA(x, gstate)

In [42]:
vha.optimize()

In [43]:
print('Minimum energy: ', vha.sol.fun)
print('Number of iterations: ', vha.sol.nfev)
print('Estimated ground state: ', np.dot(prod_time_evo(vha.ham_sets, vha.sol.x), vha.start_state))
print(vha.sol.x)

Minimum energy:  (-5.627213005295946+4.440892098500626e-16j)
Number of iterations:  24
Estimated ground state:  [[ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 7.48983787e-18+1.16647679e-17j]
 [ 9.95450485e-02+1.55032713e-01j]
 [ 1.40040298e-01+2.18100525e-01j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 1.40040298e-01+2.18100525e-01j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 2.99927365e-17+4.67110658e-17j]
 [ 0.00000000e+00+0.00000000e+00j]
 [ 1.54432946e-01+2.40515817e-01j]
 [-5.31299937

### Finding Ground State of Non-interacting Terms

VHA works because finding the ground state of non-interacting terms is efficiently computable on a QC and the VHA can evolve from that ground state to the ground state of the entire Hubbard Hamiltonian. 

We'll start by finding the ground state by using numpy to see if this actually work. 

In [106]:
non_interacting_ham = 0
for i in range(4): 
    non_interacting_ham += x.ham_sets[i]

In [123]:
non_int_ham_noise = non_interacting_ham + np.random.normal(size=(256,256))
eig_non = np.linalg.eig(non_int_ham_noise)
index = np.argmin(eig_non[0])
genergy_non = eig_non[0][index]
print('index', index)
print('Lowest energy: ', genergy_non)
gstate_non = np.transpose(NKron(eig_non[1][:,index]))
print(gstate_non.shape)
print(non_int_ham_noise.shape)

index 30
Lowest energy:  (-12.069716670621983-0.8858657097025394j)
(256, 1)
(256, 256)


In [124]:
eig_non[0][index]

(-12.069716670621983-0.8858657097025394j)

In [125]:
test_prod = np.dot(non_int_ham_noise, gstate_non)
conj = np.conjugate(gstate_non)
res = 0
for i in range(len(conj)):
    con = np.real(conj[i])
    pro = np.real(test_prod[i])
    if np.abs(con) < 1e-20: 
        continue
    elif np.abs(pro) < 1e-20: 
        continue
    else: 
        res += con * pro
#     print(con)
#     print(pro)
#     print(res)
#     print('---') 
#print('Where does the e-31 come from?!?')
res

array([-9.68754896])

In [126]:
# Choose one of the eigenvectors from lowest eigenvalue
vha = VHA(x, gstate_non)
vha.optimize()

In [127]:
vha.sol.fun

(-0.05128967378579505-6.938893903907228e-18j)

In [50]:
for ind in range(0, 80): 
    vha = VHA(x, np.transpose(NKron(eig_non[1][:,ind])))
    vha.optimize()
    print(ind, ' : ', np.real(vha.sol.fun))
    if np.real(vha.sol.fun) < -5: 
        print('Got it! Index is', ind)

0  :  3.654130282613697e-10
1  :  8.738483868305297e-11
2  :  1.7264573428332433e-07
3  :  1.3628953660177994e-08
4  :  9.419659151470211e-18
5  :  2.901359228034506e-07
6  :  2.0331787903066903e-07
7  :  3.534244842670126e-07
8  :  1.9521293394539254e-07
9  :  2.9426457479140336e-11
10  :  1.851936527660317e-07
11  :  8.553956154511155e-12
12  :  2.439738427686138e-07
13  :  3.576230985648989e-07
14  :  9.935281789499905e-10
15  :  1.569185876574495e-07
16  :  -0.09033726947176099
17  :  1.4236389823121674e-07
18  :  -1.3460160948462528e-05
19  :  -9.241050268732482e-08
20  :  -0.05581280382559988
21  :  1.618833029294761e-07


KeyboardInterrupt: 